In [ ]:
#

In [1]:
 pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [78]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import datetime

# Load the Excel file
file_path = '/Users/aswathy.n/Desktop/QuickRegressionTesting1.xlsx'
url_sheet_name = 'URLSheet'
data_sheet_name = 'DataSheet'

# Read the URL from the first sheet
df_url = pd.read_excel(file_path, sheet_name=0)
url_column = 'URL'
base_url = df_url.at[0, url_column]

# Read the input texts and expected responses from the specified sheet
df_data = pd.read_excel(file_path, sheet_name=data_sheet_name)
skuCol = 'SKU'
questionCol = 'Question'
expected_response_column = 'Expected Response'


In [79]:
print(pd.__version__)

2.2.2


In [95]:
# Load the Excel file
excel_file = pd.ExcelFile('/Users/aswathy.n/Desktop/QuickRegressionTesting5.xlsx')

# List all sheet names
sheet_names = excel_file.sheet_names
print("Sheet names:", sheet_names)

# Read the first sheet into a DataFrame
df_sheet1 = pd.read_excel(excel_file, sheet_name='URLSheet')
print("First sheet data:")
print(df_sheet1)
pdp_AIWidget_QA=df_sheet1.loc[1,'URL']

# Read the third sheet into a DataFrame
df_sheet3 = pd.read_excel(excel_file, sheet_name='QA-PDPAIWidget')
print("Third sheet data:")
print(df_sheet3.head())

Sheet names: ['URLSheet', 'Deployment_CountryList', 'MandatoryQs', 'MandatoryQ', 'Prod-PDPAIWidget', 'QA-PDPAIWidget', 'Sheet3', 'QA3-FSHS', 'QA-MagellenCitations', 'QA_Trials', 'Prod_trials', 'Sheet2']
First sheet data:
                                                 URL
0  https://www.thermofisher.com/order/catalog/pro...
1  https://www.qa.thermofisher.com/order/catalog/...
2  https://qa3.fishersci.com/us/en/customer-help-...
3  https://fishersci.com/us/en/customer-help-supp...
4  https://www.qa.thermofisher.com/antibody/produ...
Third sheet data:
                                          Question     SKU  \
0                               protein staructure  NP0006   
1                                              dna  NP0006   
2                      Certificates for 2908140000  NP0006   
3  how to detect the binding between MGP and BMP-2  NP0006   
4                    How to generate the web quote  NP0006   

                                     Expected Answer  
0  Protein Stru

In [96]:
pdp_AIWidget_QA

'https://www.qa.thermofisher.com/order/catalog/product/NP00061?SID=srch-srp-NP00061'

In [97]:
# Initialize the web driver (make sure to have the appropriate driver for your browser)
driver = webdriver.Chrome() #executable_path='/Users/aswathy.n/Documents/chromedriver')
driver.maximize_window()

In [98]:

now=datetime.datetime.now()
formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S")
print ("current date time is "+ str(formatted_date_time))

current date time is 2025-12-24 16:09:30


#This is for hard coded run
# Initialize lists to store results
matches = []
mismatches = []

# Function to calculate cosine similarity
def calculate_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    cosine_sim = cosine_similarity(vectors)
    return cosine_sim[0][1]

# Function to modify the URL
def modify_url(base_url, new_value):
    modified_url = re.sub(r'/product/[^?]+', f'/product/{new_value}', base_url)
    
    modified_url = re.sub(r'srp-[^&]+', f'srp-{new_value}', modified_url)
    modified_url="https://www.thermofisher.com/order/catalog/product/NP00061?SID=srch-srp-NP00061"
    return modified_url

# Initialize the previous URL modifier value
previous_url_modifier = None

# Open the web portal with the modified URL
modified_url="https://www.thermofisher.com/order/catalog/product/NP00061?SID=srch-srp-NP00061"
driver.get(modified_url)


time.sleep(10)

cookie_accept=driver.find_element(By.ID, 'truste-consent-button')
cookie_accept.click()
time.sleep(15)


In [99]:
# Initialize lists to store results
matches = []
mismatches = []

# Function to calculate cosine similarity
def calculate_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    cosine_sim = cosine_similarity(vectors)
    return cosine_sim[0][1]

# Function to modify the URL
def modify_url(base_url, new_value):
    modified_url = re.sub(r'/product/[^?]+', f'/product/{new_value}', base_url)
    
    modified_url = re.sub(r'srp-[^&]+', f'srp-{new_value}', modified_url)
   # modified_url="https://www.thermofisher.com/order/catalog/product/NP00061?SID=srch-srp-NP00061"
    print (modified_url, new_value)
    return modified_url

# Initialize the previous URL modifier value
previous_url_modifier = None

def set_cookie(driver, name, value):
    driver.add_cookie({'name': name, 'value': value})

# Function to delete the cookie
def delete_cookie(driver, name):
    driver.delete_cookie(name)



In [100]:
# Open the web portal with the modified URL
base_url=pdp_AIWidget_QA
country_code='US'
driver.get(base_url)

time.sleep(10)

cookie_accept=driver.find_element(By.ID, 'truste-consent-button')
cookie_accept.click()
time.sleep(15)


# Delete the existing cookie CK_ISO_CODE
delete_cookie(driver, 'CK_ISO_CODE')
#Setting new value
set_cookie(driver, 'CK_ISO_CODE', country_code)


driver.refresh()
time.sleep(10)



In [101]:
#To be included later!!!
# Iterate over the rows in the DataFrame
for index, row in df_sheet3.iterrows():
    SKU = row['SKU']
    Question = row['Question']
    expected_response = row['Expected Answer']
    print ("Question, expected_response, SKU")
    print (Question,expected_response, SKU)

    # Modify the URL only if the URL modifier value changes
    if SKU != previous_url_modifier:
        modified_url = modify_url(base_url, SKU)
        previous_url_modifier = SKU
        driver.get(modified_url)
        print (modified_url, base_url)
        time.sleep(10)
        # Click on the specific tab before entering the input text
        segment_tab = driver.find_element(By.XPATH, '/html/body/div[9]/div/div[3]/div[2]/core-navigationbar/core-navigationbar-item[2]//div') #
                                                   #  /html/body/div[9]/div/div[3]/div[2]/core-navigationbar/core-navigationbar-item[2]//div
       # segment_tab.click()     #/html/body/div[10]/div/div[2]/div[2]/core-navigationbar/core-navigationbar-item[2]//div                   
        #/html/body/div[9]/div/div[3]/div[2]/core-navigationbar/core-navigationbar-item[2]//div
        # /html/body/div[9]/div/div[2]/div[2]/core-navigationbar/core-navigationbar-item[2]//div/a 
        # Wait for the tab to load (adjust the sleep time as needed) #/html/body/div[9]/div/div[3]/div[2]/core-navigationbar/core-navigationbar-item[2]//div/a
        time.sleep(2)
    else:
        driver.get(modified_url)
        time.sleep(10)
        print ("inside else")

    # Find the input field and enter the text
    input_field = driver.find_element(By.CSS_SELECTOR, '#search-component-wrapper-ssw > core-card > div > div.input-search-wrapper-ssw > core-search')  # Adjust the locator as needed
    input_field.click()                               #  '/html/body/div[9]/div/div[3]/div[3]/div[2]/smart-search-web-component/div/core-card/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/div'
  #  input_field.clear()
    input_field.send_keys(Question)
    input_field.send_keys(Keys.RETURN)

    # Wait for the response to be generated (2 minutes)
    time.sleep(50)

    # Find the response field and get the text
    #response_field = driver.find_element(By.CLASS_NAME, 'input-search hydrated')  # Adjust the locator as needed
    #generated_response = response_field.text

   # response_field = driver.find_element(By.XPATH,'/html/body/div[9]/div/div[2]/div[3]/div[2]/smart-search-web-component/div/core-card/div/div[2]/div/div[2]') # '//*[@id="search-component-wrapper-ssw"]/core-card/div/div[2]/div/div[2]/div/div[1]/div/div/div/div')  # Adjust the locator as needed
   #  response_field = driver.find_element(By.XPATH,'/html/body/div[9]/div/div[3]/div[3]/div[2]/smart-search-web-component/div/core-card/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div')
   # response_field = driver.find_element(By.XPATH,'/html/body/div[9]/div/div[3]/div[3]/div[2]/smart-search-web-component/div/core-card/div/div[2]/div/div[2]/div/div[1]/div/div/div')
   # response_field = driver.find_element(By.CSS_SELECTOR, '#search-component-wrapper-ssw > core-card > div > div.input-search-wrapper-ssw > core-search')
    response_field = driver.find_element(By.XPATH,'/html/body/div[9]/div/div[3]/div[3]/div[2]/smart-search-web-component/div/core-card/div/div[2]/div/div[2]')
    generated_response = response_field.text       
    print("generated_response")
    print(generated_response)
                                                    #//*[@id="search-component-wrapper-ssw"]/core-card/div/div[2]/div/div[2]/div/div[1]/div/div/div/div
                                                    #//*[@id="search-component-wrapper-ssw"]/core-card/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/div
                                                    #//*[@id="search-component-wrapper-ssw"]/core-card/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div
                                                    #//*[@id="search-component-wrapper-ssw"]/core-card/div/div[2]/div[3]/div[2]/div/div[1]/div
    # Calculate cosine similarity
    similarity = calculate_cosine_similarity(generated_response, expected_response)

    # Compare the generated response with the expected response using cosine similarity
    if similarity >= 0.60:  # Threshold for considering a match (adjust as needed)
        matches.append((SKU,Question ,generated_response, expected_response, similarity))
    else:
        mismatches.append((SKU,Question ,generated_response, expected_response, similarity))

# Close the web driver
#driver.quit()

print(matches)
print(mismatches)

Question, expected_response, SKU
protein staructure Protein Structure Information
The molecular weight of a protein can be determined based upon its relative mobility by constructing a standard curve with protein standards of known molecular weights. The protein mobility in SDS-PAGE gels is dependent on the:
Length of the protein in its fully denatured state,
SDS-PAGE buffer systems, and
Secondary structure of the protein.
The same molecular weight standard may have slightly different mobility, resulting in different apparent molecular weight when run in different SDS-PAGE buffer systems.
When using SDS-PAGE for molecular weight determination, slight deviations from the calculated molecular weight of a protein (calculated from the known amino acid sequence) can occur due to the retention of varying degrees of secondary structure in the protein, even in the presence of SDS. This phenomenon is observed in highly organized secondary structures (collagens, histones, or highly hydrophobic m

In [102]:
matches

[('NP0006',
  'Certificates for 2908140000',
  "Certificates (5 matches)\nLot #\nCertificate Type\nDate\nCatalog Number(s)\nTFRSRE26S\nCertificate of Analysis\nNov 28, 2025\nNP0006\nTFRSK6JNH\nCertificate of Analysis\nNov 28, 2025\nNP0006\nTFQALWUGZ\nCertificate of Analysis\nNov 28, 2025\nNP0006\nTFQAV4VT9\nCertificate of Analysis\nNov 28, 2025\nNP0006\nTFQARNCPC\nCertificate of Analysis\nNov 19, 2025\nNP0006\nHere are the 5 latest certificate(s). If you can't find the one you're looking for, please provide the LOT number below.\nGive feedback",
  "Certificates (5 matches)\nLot #\nCertificate Type\nDate\nCatalog Number(s)\n3268014\nCertificate of Analysis\nSep 08, 2025\nNP00061\nTFRS1ZOMP\nCertificate of Analysis\nSep 03, 2025\nNP0006\nTFQAMOLFB\nCertificate of Analysis\nSep 02, 2025\nNP0006\nTFRSHBWU4\nCertificate of Analysis\nAug 31, 2025\nNP0006\nTFRSWNZXP\nCertificate of Analysis\nAug 30, 2025\nNP0006\nHere are the 5 latest certificate(s). If you can't find the one you're looking f

In [103]:
mismatches

[('NP0006',
  'protein staructure',
  'Related Products for Protein Structure with SKU np0006\nThe product with catalog number np0006 is the NuPAGE Transfer Buffer (20X), used for transferring proteins from NuPAGE Bis-Tris and NuPAGE Tris-Acetate gels to membranes in western blotting. It maintains a neutral pH environment that protects protein integrity during transfer.\nRelated products that support protein structure analysis and electrophoresis include:\nProduct Application Catalog Number\nNuPAGE Antioxidant Enhances transfer of reduced proteins to membranes, preserving protein integrity NP0005\nNuPAGE LDS Sample Buffer (4X) Sample buffer recommended for SDS-PAGE, preserves protein structure NP0007\nNuPAGE Sample Reducing Agent (10X) Reduces disulfide bonds in proteins for denaturation NP0009\nHiMark Pre-Stained High Molecular Weight Protein Standard Protein molecular weight standard for calibration LC5699\nNovex Sharp Pre-stained Protein Standard Protein molecular weight standard fo

In [104]:

now=datetime.datetime.now()
formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S")
print ("current date time is "+ str(formatted_date_time))

current date time is 2025-12-24 17:09:46


In [105]:
matches

[('NP0006',
  'Certificates for 2908140000',
  "Certificates (5 matches)\nLot #\nCertificate Type\nDate\nCatalog Number(s)\nTFRSRE26S\nCertificate of Analysis\nNov 28, 2025\nNP0006\nTFRSK6JNH\nCertificate of Analysis\nNov 28, 2025\nNP0006\nTFQALWUGZ\nCertificate of Analysis\nNov 28, 2025\nNP0006\nTFQAV4VT9\nCertificate of Analysis\nNov 28, 2025\nNP0006\nTFQARNCPC\nCertificate of Analysis\nNov 19, 2025\nNP0006\nHere are the 5 latest certificate(s). If you can't find the one you're looking for, please provide the LOT number below.\nGive feedback",
  "Certificates (5 matches)\nLot #\nCertificate Type\nDate\nCatalog Number(s)\n3268014\nCertificate of Analysis\nSep 08, 2025\nNP00061\nTFRS1ZOMP\nCertificate of Analysis\nSep 03, 2025\nNP0006\nTFQAMOLFB\nCertificate of Analysis\nSep 02, 2025\nNP0006\nTFRSHBWU4\nCertificate of Analysis\nAug 31, 2025\nNP0006\nTFRSWNZXP\nCertificate of Analysis\nAug 30, 2025\nNP0006\nHere are the 5 latest certificate(s). If you can't find the one you're looking f

In [106]:
mismatches

[('NP0006',
  'protein staructure',
  'Related Products for Protein Structure with SKU np0006\nThe product with catalog number np0006 is the NuPAGE Transfer Buffer (20X), used for transferring proteins from NuPAGE Bis-Tris and NuPAGE Tris-Acetate gels to membranes in western blotting. It maintains a neutral pH environment that protects protein integrity during transfer.\nRelated products that support protein structure analysis and electrophoresis include:\nProduct Application Catalog Number\nNuPAGE Antioxidant Enhances transfer of reduced proteins to membranes, preserving protein integrity NP0005\nNuPAGE LDS Sample Buffer (4X) Sample buffer recommended for SDS-PAGE, preserves protein structure NP0007\nNuPAGE Sample Reducing Agent (10X) Reduces disulfide bonds in proteins for denaturation NP0009\nHiMark Pre-Stained High Molecular Weight Protein Standard Protein molecular weight standard for calibration LC5699\nNovex Sharp Pre-stained Protein Standard Protein molecular weight standard fo

In [107]:
matchingResults="matches_report_QAPDP_"+country_code+"_"+str(now.strftime("%Y-%m-%d %H_%M_%S"))+".csv"
mismatchingResults="mismatches_report_QAPDP_"+country_code+"_"+str(now.strftime("%Y-%m-%d %H_%M_%S"))+".csv"

In [108]:
matchingResults, mismatchingResults

('matches_report_QAPDP_US_2025-12-24 17_09_46.csv',
 'mismatches_report_QAPDP_US_2025-12-24 17_09_46.csv')

In [109]:
import datetime
# Generate the report
matches_report = pd.DataFrame(matches, columns=["SKU","Question", 'Generated Response', 'Expected Response', 'Cosine Similarity'])
matches_report.to_csv(matchingResults, index=False)

mismatches_report = pd.DataFrame(mismatches, columns=["SKU","Question", 'Generated Response', 'Expected Response', 'Cosine Similarity'])
mismatches_report.to_csv(mismatchingResults, index=False)

print("Report generated successfully.")

Report generated successfully.


# Generate the report
matches_report = pd.DataFrame(matches, columns=["SKU","Question", 'Generated Response', 'Expected Response', 'Cosine Similarity'])
matches_report.to_excel('matches_report_QA.xlsx', index=False)

mismatches_report = pd.DataFrame(mismatches, columns=["SKU","Question", 'Generated Response', 'Expected Response', 'Cosine Similarity'])
mismatches_report.to_excel('mismatches_report_QA.xlsx', index=False)

print("Report generated successfully.")